In [1]:
import numpy as np 
import tensorflow as tf 
from keras import layers 

2026-01-03 13:45:47.007622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class sampling(layers.Layer):
    def call(self,inputs):
        mean,log_var=inputs 
        batch=tf.shape(mean)[0]
        dim=tf.shape(mean)[1]
        epsilon=tf.random.normal(shape(batch,dim))
        return mean + tf.exp(0.5*log_var)+epsilon